In [1]:
%autocall 2

Automatic calling is: Full


In [2]:
%matplotlib -l

Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'gtk4', 'wx', 'qt4', 'qt5', 'qt6', 'qt', 'osx', 'nbagg', 'webagg', 'notebook', 'agg', 'svg', 'pdf', 'ps', 'inline', 'ipympl', 'widget']


In [3]:
%matplotlib tk

In [4]:
from datetime import datetime

import matplotlib as mpl

# mpl.use('TkAgg')

import matplotlib.pyplot as plt
import numpy as np

In [5]:
import ipython_memory_usage

In [6]:
%ipython_memory_usage_start

'memory profile enabled'

In [6] used 0.0195 MiB RAM in 0.35s, peaked 0.00 MiB above current, total RAM usage 106.17 MiB


In [7]:
t0 = 0

dtmin = np.inf
dtmax = -np.inf
dtmean = 0
ndt = 0

def update_title(axes, title):
    global t0, dtmin, dtmax, dtmean, ndt

    if not t0:
        t0 = datetime.now()
        t1 = t0
    else:
        t1 = datetime.now()
        dt = t1 - t0
        dtmin = np.min([dt.microseconds, dtmin])
        dtmax = np.max([dt.microseconds, dtmax])
        dtmean += dt.microseconds
        ndt += 1
        t0 = t1
    
    axes.figure.canvas.restore_region(bg)
    
    title.set_text(t1)
    axes.draw_artist(title)
    
    axes.figure.canvas.blit(fig.bbox)
    axes.figure.canvas.flush_events()
    # slow
    # axes.figure.canvas.draw()

In [7] used 0.1211 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 106.29 MiB


In [8]:
def on_close(event):
    global dtmin, dtmax, dtmean, ndt
    
    print("closed figure")
    
    print(dtmean / ndt / 1000, dtmin / 1000, dtmax / 1000)

In [8] used 0.0078 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 106.30 MiB


In [9]:
plt.ioff()
plt.interactive(False)
plt.isinteractive()

False

In [9] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 106.30 MiB


In [10]:
plt.close('all')

In [10] used 0.0078 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 106.31 MiB


In [11]:
try:
    plt.close(fig = fig)
    del fig, ax
except:
    pass
    
fig, ax = plt.subplots()

x = np.linspace(-3, 3)
ax.plot(x, x ** 2)

title = ax.set_title(datetime.now(), animated = True)

fig.canvas.mpl_connect('close_event', on_close)

# pause causes a full draw
# plt.pause(0.1)
fig.canvas.draw()

In [11] used 8.1133 MiB RAM in 0.27s, peaked 0.00 MiB above current, total RAM usage 114.42 MiB


In [12]:
bg = fig.canvas.copy_from_bbox(fig.bbox)

ax.draw_artist(title)

fig.canvas.blit(fig.bbox)

In [12] used 1.1875 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 115.61 MiB


In [13]:
# Create a new timer object. Set the interval to 100 milliseconds
timer = fig.canvas.new_timer(interval = 0) # 0: as fast as possible, here measured about 3000 microseconds!!! Wow! 333 frames per second
# (1000 is default) and tell the timer what function should be called.

print("timer interval: ", timer.interval)

timer.add_callback(update_title, ax, title)

# timer.start()

# Or could start the timer on first figure draw:
def start_timer(event):
    timer.start()
    fig.canvas.mpl_disconnect(drawid)
drawid = fig.canvas.mpl_connect('draw_event', start_timer)

timer interval:  1
In [13] used 0.0156 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 115.62 MiB


In [14]:
plt.show(block = True)

closed figure
4.972470525187567 3.927 66.838
In [14] used 1.5156 MiB RAM in 4.80s, peaked 1.14 MiB above current, total RAM usage 117.14 MiB


In [15]:
timer.stop()

In [15] used 0.3867 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 105.67 MiB


In [18]:
print(dtmean / ndt / 1000, dtmin / 1000, dtmax / 1000)

9.022122093023256 0.0 23.96
In [18] used 0.0781 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 105.75 MiB


In [16]:
plt.close(fig = fig)

In [16] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 105.67 MiB


In [17]:
plt.close('all')

In [17] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 105.67 MiB


In [1]:
class BlitManager:
    def __init__(self, canvas, animated_artists=()):
        """
        Parameters
        ----------
        canvas : FigureCanvasAgg
            The canvas to work with, this only works for subclasses of the Agg
            canvas which have the `~FigureCanvasAgg.copy_from_bbox` and
            `~FigureCanvasAgg.restore_region` methods.

        animated_artists : Iterable[Artist]
            List of the artists to manage
        """
        
        self.canvas = canvas
        self._bg = None
        self._artists = []

        for a in animated_artists:
            self.add_artist(a)
        
        # grab the background on every draw
        self.cid = canvas.mpl_connect("draw_event", self.on_draw)

    def on_draw(self, event):
        """Callback to register with 'draw_event'."""
        canvas = self.canvas
        
        if event is not None:
            if event.canvas != canvas:
                raise RuntimeError
        
        self._bg = canvas.copy_from_bbox(canvas.figure.bbox)
        
        self._draw_animated()

    def add_artist(self, art):
        """
        Add an artist to be managed.

        Parameters
        ----------
        art : Artist

            The artist to be added.  Will be set to 'animated' (just
            to be safe).  *art* must be in the figure associated with
            the canvas this class is managing.

        """
        if art.figure != self.canvas.figure:
            raise RuntimeError
        
        art.set_animated(True)
        
        self._artists.append(art)

    def _draw_animated(self):
        """Draw all of the animated artists."""
        
        fig = self.canvas.figure
        
        for a in self._artists:
            fig.draw_artist(a)

    def update(self):
        """Update the screen with animated artists."""
        
        canvas = self.canvas
        
        fig = canvas.figure
        
        # paranoia in case we missed the draw event,
        if self._bg is None:
            self.on_draw(None)
        else:
            # restore the background
            canvas.restore_region(self._bg)
            
            # draw all of the animated artists
            self._draw_animated()
            
            # update the GUI state
            canvas.blit(fig.bbox)
        
        # let the GUI event loop process anything it has to do
        canvas.flush_events()

In [ ]:
# make a new figure
fig, ax = plt.subplots()

# add a line
(ln,) = ax.plot(x, np.sin(x), animated = True)

# add a frame number
fr_number = ax.annotate("0", (0, 1),
                        xycoords = "axes fraction", xytext = (10, -10), textcoords = "offset points",
                        ha = "left", va = "top",
                        animated = True)

blitmanager = BlitManager(fig.canvas, [ln, fr_number])

# make sure our window is on the screen and drawn
plt.show(block = False)

# for a draw event with pause
plt.pause(0.1)

for j in range(100):
    # update the artists
    ln.set_ydata(np.sin(x + (j / 100) * np.pi))
    
    fr_number.set_text(f"frame: {j}")
    
    # tell the blitting manager to do its thing
    blitmanager.update()

In [ ]:
import queue, threading

In [ ]:
q = queue.Queue(maxsize = 10)

In [ ]:
event = threading.Event

In [ ]:
event.wait()